In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import PIL
import nltk
#nltk.download('vader_lexicon')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("/Users/andrewcohen/Desktop/KaggleData"))

# Any results you write to the current directory are saved as output.

['test_images', '.Rhistory', 'InstallPackages.R', 'breed_labels.csv', '.DS_Store', 'test', 'train_metadata', 'color_labels.csv', 'test_sentiment', 'test_metadata', 'GitStuff', 'README.md', 'train_sentiment', 'pet-adoption-eda-and-preliminary-model.Rmd', 'train.csv', '.git', 'train_images', 'state_labels.csv']


In [4]:
train_df = pd.read_csv("/Users/andrewcohen/Desktop/KaggleData/train.csv")
test_df = pd.read_csv("/Users/andrewcohen/Desktop/KaggleData/test/test.csv")
breeds_df = pd.read_csv("/Users/andrewcohen/Desktop/KaggleData/breed_labels.csv")
dog_cat = {1: 'dog', 2: 'cat'}

In [5]:
def mix_breed(row):
    b1 = row['Breed1']
    b2 = row['Breed2']
    if any([b1 == 307, b2 == 307]) or all([b1 > 0, b2 > 0, b1 != b2]):
        return 1
    else:
        return 0
    
def new_breed1(row):
    b1 = row['Breed1']
    b2 = row['Breed2']
    if b1 == b2:
        if b1 in [0, 307]:
            return -1
        else:
            return b1       
    elif b1 == 307 or b2 == 307:
        if b1 == 307 and b2 != 0:
            return b2
        elif b2 == 0:
            return -1
        else:
            return b1 if b1 != 0 else -1
    else:
        if b1 == 0:
            return b2
        else:
            return b1
        
def new_breed2(row):
    b1 = row['Breed1']
    b2 = row['Breed2']
    if b1 == b2:
        return -1 if b1 in [0, 307] else 0      
    elif b1 == 307 or b2 == 307:
        return -1
    else:
        if b1 !=0  and b2 != 0:
            return b2
        else:
            return 0

train_df['Mixed'] = train_df.apply(lambda row: mix_breed(row), axis=1)
train_df['NewBreed1'] = train_df.apply(lambda row: new_breed1(row), axis=1)
train_df['NewBreed2'] = train_df.apply(lambda row: new_breed2(row), axis=1)
test_df['Mixed'] = test_df.apply(lambda row: mix_breed(row), axis=1)
test_df['NewBreed1'] = test_df.apply(lambda row: new_breed1(row), axis=1)
test_df['NewBreed2'] = test_df.apply(lambda row: new_breed2(row), axis=1)

def has_name(name):
    if 'no name' in name.lower():
        return 0
    else:
        return 1
    
train_df['HasName'] = train_df.apply(lambda row: has_name(str(row)), axis=1)
test_df['HasName'] = test_df.apply(lambda row: has_name(str(row)), axis=1)

def get_top_labann(lab_anns):
    scores = np.array([a['score'] for a in lab_anns])
    tops = scores.argsort()
    return {'top_annotation': lab_anns[tops[-1]]['description'], 'annot_score': lab_anns[tops[-1]]['score']}

def get_image_size(im):
    return {'im_w': im.size[0], 'im_d': im.size[1], '#_pix': np.product(im.size)}

def get_im_meta(petID, df, sample):
    if sample == 'train':
        fp_p = "../input/train_images/"
        fp_j = "../input/train_metadata/"
    else:
        fp_p = "../input/test_images/"
        fp_j = "../input/test_metadata/"
    pet_type = int(df[df['PetID']==petID]['Type'])
    possible_breeds = set([x.lower() for x in breeds_df[breeds_df['Type']==pet_type]['BreedName'].values])
    try:
        with open(fp_j+petID+'-1.json', 'r', encoding='iso-8859-1') as im_meta:
            im_data_m = json.loads(im_meta.read())
        img = PIL.Image.open(fp_p+petID+'-1.jpg')
        res = {'PetID': petID}
        try:
            lab_anns = get_top_labann(im_data_m['labelAnnotations'])
            lab_a = lab_anns['top_annotation'].lower()
            lab_anns['top_annotation'] = int(any([lab_a in possible_breeds, dog_cat[pet_type] in lab_a]))
        except KeyError:
            lab_anns = {'top_annotation': -1, 'annot_score': -1}
        for x in lab_anns:
            res[x] = lab_anns[x]
        im_s = get_image_size(img)
        for x in im_s:
            res[x] = im_s[x]
        return res
    except FileNotFoundError:
        return {'PetID': petID, 'im_w': -1, 'im_d': -1, '#_pix': -1,  'top_annotation': -1, 'annot_score': -1}
    
meta_data_train = []
for petID in train_df['PetID'].values:
    meta_data_train.append(get_im_meta(petID, train_df, 'train'))
meta_df_train = pd.DataFrame(meta_data_train)
cols_to_use = train_df.columns.difference(meta_df_train.columns)
train_df = pd.concat([train_df[cols_to_use], meta_df_train], axis=1)

meta_data_test = []
for petID in test_df['PetID'].values:
    meta_data_test.append(get_im_meta(petID, test_df, 'test'))
meta_df_test = pd.DataFrame(meta_data_test)
cols_to_use = test_df.columns.difference(meta_df_test.columns)
test_df = pd.concat([test_df[cols_to_use], meta_df_test], axis=1)

In [6]:
#Sentiment Analysis    
from nltk.sentiment import vader
vader_sent = vader.SentimentIntensityAnalyzer()

name_sent_tr = []
for name in train_df["Name"].values:
    if has_name(str(name)):
        name_sent_tr.append(vader_sent.polarity_scores(str(name)))
    else:
        name_sent_tr.append({'compound': 0, 'neg': 0.0, 'neu': 0.0, 'pos': 0})
        
name_sent_ts = []
for name in test_df["Name"].values:
    if has_name(str(name)):
        name_sent_ts.append(vader_sent.polarity_scores(str(name)))
    else:
        name_sent_ts.append({'compound': 0, 'neg': 0.0, 'neu': 0.0, 'pos': 0})
        

name_feat_tr = pd.DataFrame(name_sent_tr)
name_feat_ts = pd.DataFrame(name_sent_ts)

train_df = pd.concat((name_feat_tr, train_df), axis=1)
test_df = pd.concat((name_feat_ts, test_df), axis=1)

/Users/andrewcohen/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [1]:
X = train_df[['Type', 'Age', 'NewBreed1', 'NewBreed2', 'Mixed', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'HasName', 'compound',
       'neg', 'neu', 'pos', 'VideoAmt', 'PhotoAmt','top_annotation', 'annot_score', 'im_w',
       'im_d', '#_pix']].values
y = train_df['AdoptionSpeed'].values



X_tr = test_df[['Type', 'Age', 'NewBreed1', 'NewBreed2', 'Mixed', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'HasName', 'compound',
       'neg', 'neu', 'pos', 'VideoAmt', 'PhotoAmt', 'top_annotation', 'annot_score', 'im_w',
       'im_d', '#_pix']].values

from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb

#clf = GradientBoostingClassifier(n_estimators=500, learning_rate=0.07, max_features=1,
#                                max_depth=4)
#clf.fit(X,y)

#test_df['AdoptionSpeed'] = clf.predict(X_tr)

lgb_train = lgb.Dataset(X, y)
params = {
        'objective': 'multiclass',
        'num_class': 5,
        'boosting': 'gbdt',
        'learning_rate': 0.01 ,
        'verbose': 0,
        'num_leaves': 50,
        'n_estimators': 700,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 1,
        'feature_fraction_seed': 1,
        'max_bin': 512,
        'metric' : 'multi_error'
    }

clf = lgb.train(params, train_set = lgb_train, verbose_eval=100)
test_df['AdoptionSpeed'] = clf.predict(X_tr).argmax(axis=1)

test_df[['PetID','AdoptionSpeed']].to_csv('submission.csv', index=False)

NameError: name 'train_df' is not defined